# Imports

In [1]:
import pandas as pd
import sqlite3

# Data Loading

In [2]:
conn = sqlite3.connect("../raw_data/ellipsys_test_db.sqlite")
query = "SELECT * FROM oa_trf_src"

df = pd.read_sql(query, conn, index_col='id')

## First Lines

In [3]:
df.head()

,trf,tgtTb,tgtLab,srcTb,srcLab,impact
id,,,,,,
bicprd.fnr_cmn_j,bicprd.fnr_cmn_j.set_valcode_frs_j,vld.tfnr974,rf_cle_cod,dual.sfnr974,nextval,1
bicprd.set_valcode_frs,bicprd.set_valcode_frs.set_valcode_frs,vld.tfnr974,rf_cle_cod,dual.sfnr974,nextval,1
bicprd.spik_lstvaleurs,bicprd.spik_lstvaleurs.updlstval_tpfm7a,bicprd.lstvaleurs,owner,ods.tcifp47,cd_typ_cha,-1
bicprd.spik_lstvaleurs,bicprd.spik_lstvaleurs.updlstval_tpfm7a,bicprd.lstvaleurs,tbl,bicprd.lstvaleurs,valeur,-1
bicprd.spik_lstvaleurs,bicprd.spik_lstvaleurs.updlstval_tpfm7a,bicprd.lstvaleurs,bo_saisie_auto,ods.tcif432,cd_typ_cha,-1


## Table Shape and types

In [4]:
df.shape

(500000, 6)

In [5]:
df = df.astype({"impact": int})

In [6]:
df.dtypes

trf       object
tgtTb     object
tgtLab    object
srcTb     object
srcLab    object
impact     int64
dtype: object

## Unique and Null Values

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 500000 entries, bicprd.fnr_cmn_j to spiprocs.frm2alim_dm
Data columns (total 6 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   trf     500000 non-null  object
 1   tgtTb   500000 non-null  object
 2   tgtLab  500000 non-null  object
 3   srcTb   500000 non-null  object
 4   srcLab  500000 non-null  object
 5   impact  500000 non-null  int64 
dtypes: int64(1), object(5)
memory usage: 26.7+ MB


In [8]:
df.nunique()

trf        421
tgtTb      366
tgtLab    4054
srcTb      634
srcLab    4138
impact       2
dtype: int64

## Data Transformation

In [9]:
#sqlite query

conn = sqlite3.connect("../raw_data/ellipsys_test_db.sqlite")
query = "SELECT * FROM oa_trf_src"

df = pd.read_sql(query, conn)

#Source Database
oa_trf_src = df.astype({"impact": int})
oa_trf_src_red = df.astype({"impact": int})

In [10]:
oa_trf_src.head()

,id,trf,tgtTb,tgtLab,srcTb,srcLab,impact
0,bicprd.fnr_cmn_j,bicprd.fnr_cmn_j.set_valcode_frs_j,vld.tfnr974,rf_cle_cod,dual.sfnr974,nextval,1
1,bicprd.set_valcode_frs,bicprd.set_valcode_frs.set_valcode_frs,vld.tfnr974,rf_cle_cod,dual.sfnr974,nextval,1
2,bicprd.spik_lstvaleurs,bicprd.spik_lstvaleurs.updlstval_tpfm7a,bicprd.lstvaleurs,owner,ods.tcifp47,cd_typ_cha,-1
3,bicprd.spik_lstvaleurs,bicprd.spik_lstvaleurs.updlstval_tpfm7a,bicprd.lstvaleurs,tbl,bicprd.lstvaleurs,valeur,-1
4,bicprd.spik_lstvaleurs,bicprd.spik_lstvaleurs.updlstval_tpfm7a,bicprd.lstvaleurs,bo_saisie_auto,ods.tcif432,cd_typ_cha,-1


## Factorizing

In [11]:
correspondances = {}
for column in oa_trf_src_red.columns.drop(['impact']):
    codes, uniques = pd.factorize(oa_trf_src_red[column])
    oa_trf_src_red[column] = codes
    correspondances[f'oa_trf_src_{column}_lkp'] = pd.DataFrame(uniques, columns=["champ"])

In [12]:
oa_trf_src_red.head()

,id,trf,tgtTb,tgtLab,srcTb,srcLab,impact
0,0,0,0,0,0,0,1
1,1,1,0,0,0,0,1
2,2,2,1,1,1,1,-1
3,2,2,1,2,2,2,-1
4,2,2,1,3,3,1,-1


In [13]:
oa_trf_src_red.to_sql(name='oa_trf_src_red', con=conn, index=False)

for key, table in correspondances.items():
    table.to_sql(name=key, con=conn, index_label="id")